In [1]:
import numpy as np
import pandas as pd
#documents = "I come from China and i love my hometown Guangdong"
documents = ("One day, a monkey rides his bike near the river. This time he sees a lion under a tree."\
"The lion runs at him. He is afraid and falls into the river. He can’t swim. He shouts. "\
"The rabbit hears him. He jumps into the river. The rabbit swims to the monkey, but he can’t help him. Luckily, an elephant comes along."\
" He is very strong. He helps the rabbit and monkey. Three friends are very happy. They go to the elephant’s home."\
" Then, three of them become good friends."
)

In [2]:
len(documents)

463

In [3]:
data = np.array(documents.split())
news_df = pd.DataFrame({'document':documents}, index=[0])
news_df

,document
0,"One day, a monkey rides his bike near the rive..."


In [4]:
news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z#]", " ")#去除字母以外的所有标点符号
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))#去除短句
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())#把所有数据转为小写

In [5]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords#导入停止词包，删除体制词
stop_words = stopwords.words('english')
tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split())#标记化，删除停止词

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [6]:
detokenized_doc = []#解除标记,数据到这不正常
#转换为字符串
for i in range(len(news_df)):
 t = ' '.join(tokenized_doc[i])
 detokenized_doc.append(t)

In [7]:
news_df['clean_doc'] = detokenized_doc#得到新词
news_df['clean_doc']


0    monkey rides bike near river this time sees li...
Name: clean_doc, dtype: object

In [8]:
#文档转为词项矩阵
from sklearn.feature_extraction.text import TfidfVectorizer
#max_df表示当前要忽略的文本占比
vectorizer = TfidfVectorizer(stop_words='english',max_features =1000,max_df = 1,smooth_idf = True)
X = vectorizer.fit_transform(news_df['clean_doc'])#模型适应


In [9]:
n = X.shape[-1]
X.shape

(1, 28)

In [10]:
from sklearn.decomposition import TruncatedSVD#导入Truncated分解词义矩阵
#SVD表现文档和向量主题
#n_compents必须小于当前文本的最大范围
svd_model = TruncatedSVD(n_components=n-1, algorithm='randomized', n_iter=100, random_state=122)
svd_model.fit(X)

/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_truncated_svd.py:194: RuntimeWarning: invalid value encountered in true_divide
  self.explained_variance_ratio_ = exp_var / full_var


TruncatedSVD(algorithm='randomized', n_components=27, n_iter=100,
             random_state=122, tol=0.0)

In [11]:
len(svd_model.components_)#处理后的文档数据长度

1

In [13]:
terms = vectorizer.get_feature_names()

for i, comp in enumerate(svd_model.components_):
  terms_comp = zip(terms, comp)
  sorted_terms = sorted(terms_comp, key=lambda x:x[1], reverse=True)[:7]
  print("result:")
  for t in sorted_terms:
    print(t[0],end=" ")
  print()


result:
monkey rabbit river elephant friends lion bike 
